In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 7419 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7419 kB]
Fetched 7419 kB in 1s (5757 kB/s)
debconf: delaying package configuration, since apt-ut

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertConfig
from transformers import BertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("mlewand/PROTBERT")
"""
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=512,
    hidden_size=256,
    num_hidden_layers=8,
    num_attention_heads=8,
    type_vocab_size=1,
)
model = BertForMaskedLM(config=config)
"""
model = BertForMaskedLM.from_pretrained("mlewand/PROTBERT_LM-L8_hid512")

In [5]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.mask_token_id)
print(tokenizer.pad_token_id)

0
2
4
1


# Imports

In [6]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter

import torch

# Hyperparameters

In [7]:
file_id = '1w3IQMK3PmXH-Bq6Lt_P8wxGPvr5olrZT'
destination = 'pfam.zip'
hf_name = "t5-small"
n_families_of_interest = 40
data_dirpath = "pfam"
aminoacid_separate_by = ""
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

# Utils

In [8]:
def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

In [9]:
# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm.tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")

# Get the Dataset

In [10]:
download_file_from_google_drive(file_id, destination)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(".")
!mv random_split pfam

written : 468.781 MB: : 15780it [00:11, 1429.12it/s]




saved all the data to pfam.zip. total size : 493.095 MB


# Read Data

In [11]:
def read_all_shards(partition='dev', data_dir = data_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    
    return pd.concat(shards)

def read_all_data_initial():
  global train, test, dev, all_train_ds_size, all_test_ds_size, all_dev_ds_size

  test = read_all_shards('test')
  dev = read_all_shards('dev')
  train = read_all_shards('train')

  partitions = {'test': test, 'dev': dev, 'train': train}
  for name, df in partitions.items():
      print('Dataset partition "%s" has %d sequences' % (name, len(df)))

  all_train_ds_size = len(train)
  all_test_ds_size = len(test)
  all_dev_ds_size = len(dev)

  train.reset_index(inplace=True, drop=True)
  dev.reset_index(inplace=True, drop=True)
  test.reset_index(inplace=True, drop=True)

def get_cumulative(data):
    counter = Counter(data['family_accession'])
    print(f"how many labels : {len(counter)}")
    
    datasetSize = len(data)
    xs = []
    x_labels = []
    ys = []

    t = 0
    cumulative = []

    for i,(x, y) in  enumerate(counter.most_common()):
        xs.append(i)
        x_labels.append(x)
        ys.append(y)
        t += y / datasetSize
        cumulative.append(t)
    return cumulative


# EXECUTION CODE
print('Available dataset partitions: ', os.listdir(data_dirpath))
read_all_data_initial()
cumulative = get_cumulative(train)
print(f"{n_families_of_interest} classes is {100 * round( cumulative[n_families_of_interest-1],3)} portion of training data")

familiesOfInterest = train.family_accession.value_counts()[:n_families_of_interest]

mask = train.family_accession.isin(familiesOfInterest.index.values)
train = train.loc[mask,:]

mask = dev.family_accession.isin(familiesOfInterest.index.values)
dev = dev.loc[mask,:]

mask = test.family_accession.isin(familiesOfInterest.index.values)
test = test.loc[mask,:]


################################################################################
train_seq = train['sequence']
dev_seq = dev['sequence']
test_seq = test['sequence']

################################################################################
train_sentences = train_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
validation_sentences = dev_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
test_sentences = test_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))

################################################################################
train_labels = train['family_accession'].apply(lambda x: x.split('.')[0])
validation_labels = dev['family_accession'].apply(lambda x: x.split('.')[0])
test_labels = test['family_accession'].apply(lambda x: x.split('.')[0])

Available dataset partitions:  ['train', 'test', 'random_split', 'dev']
Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences
how many labels : 17929
40 classes is 4.3 portion of training data


In [12]:
train_sentences

10                                      SLKFLNFAQNEFNGSIPESV
37         KRVIDIVFSIIGIVVLSPLMLAICIYIKISSKGNAIFSHIRLGYRG...
62         LGYWPVLAGRVVAGDQRGRQLGFPTANLAVPPELKLPAYGVYACLA...
103        SGAAIAALFAGMGSAKGTSIAGQAAAGVVTEDPSKFGQLLVLQLLP...
116        FVLGLVAGSLSFGGAYTAIPFVQVEAVLKGGWLPQSVFIDCIAIGN...
                                 ...                        
1086628    ILDGNATAAAIKEELKERVEALKAKGITPGLGTVLVGDDPASRSYV...
1086658    TGLIQDLGRVERVIPGGMTDLWIHTSLGAAAFELGESIAVNGACLT...
1086682    CETCKGSGGREKAKPHDCSVCGGRGQTLRIQTVGGMLSQVTTQCAN...
1086684    LNAVKLLARVAETRSFTQAAVSLGLTQSGLSRAIGRLEAELGVRLL...
1086720    VLDIGCGNGKITAQIAERVPAGAVLGIDSSEQMISFASSHFAPADW...
Name: sequence, Length: 46902, dtype: object

In [13]:
labels = set()

for v in tqdm.tqdm(train_labels):
  labels.add(v)

label_to_id = {k:v for v,k in enumerate(labels)}

print()
print(len(labels))

100%|██████████| 46902/46902 [00:00<00:00, 1806489.12it/s]


40


In [14]:
from datasets import Dataset
dataset = Dataset.from_dict({"sentence": train_sentences})
dataset_val = Dataset.from_dict({"sentence": validation_sentences})

In [15]:
print(dataset)
print(dataset_val)

Dataset({
    features: ['sentence'],
    num_rows: 46902
})
Dataset({
    features: ['sentence'],
    num_rows: 5841
})


In [16]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint)
print(f"datapoint : {datapoint}, len({len(datapoint)})")
print(encoded)

datapoint : SLKFLNFAQNEFNGSIPESV, len(20)
[0, 55, 48, 47, 42, 48, 50, 42, 37, 53, 50, 41, 42, 50, 43, 55, 45, 52, 41, 55, 58, 2]


In [17]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [18]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset_val = dataset_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/46902 [00:00<?, ? examples/s]

Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [19]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.05)
tok = tokenizer(train_sentences.iloc[0], truncation=True)
data_collator([tok])

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[ 0, 55, 48, 47, 42, 48, 50, 42,  4, 53, 50, 41, 42, 50, 43, 55, 45, 52,
         41, 55, 58,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100, -100, -100, -100, -100, -100, -100, -100,   37, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]])}

In [20]:
def compute_metrics(pred):
  labels = pred.label_ids
  # labels = np.argmax(labels, axis=1)
  preds = pred.predictions

  tot = np.sum(labels != -100) + 1e-10
  classified_correctly = 0
  for i in range(preds.shape[0]):
    mask = preds[i, :] != -100
    classified_correctly += np.sum(preds[i, mask] == labels[i, mask])
  
  return {"exact_matches": classified_correctly / tot}

In [21]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits, dim=-1)
    return pred_ids

In [22]:
!pip install torchinfo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [23]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                  Param #
BertForMaskedLM                                         --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              66,816
│    │    └─Embedding: 3-2                              131,072
│    │    └─Embedding: 3-3                              256
│    │    └─LayerNorm: 3-4                              512
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             14,723,072
├─BertOnlyMLMHead: 1-2                                  --
│    └─BertLMPredictionHead: 2-3                        --
│    │    └─BertPredictionHeadTransform: 3-7            66,304
│    │    └─Linear: 3-8                                 67,077
Total params: 15,055,109
Trainable params: 15,055,109
Non-trainable params: 0

In [24]:
from transformers import Trainer, TrainingArguments

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.05)

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=1e-4,
    num_train_epochs=200,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True,
    push_to_hub=True,
    hub_model_id="mlewand/PROTBERT_LM-L8_hid512",
    eval_accumulation_steps = 8 # Transfer GPU tensors during validation to RAM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    preprocess_logits_for_metrics= preprocess_logits_for_metrics
)

trainer.train()

Cloning https://huggingface.co/mlewand/PROTBERT_LM-L8_hid512 into local empty directory.


Download file pytorch_model.bin:   0%|          | 7.40k/57.2M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/57.2M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Exact Matches
1,2.564100,2.555925,0.208697
2,2.550600,2.530543,0.212246
3,2.527900,2.515243,0.221396
4,2.509000,2.488445,0.234904
5,2.486500,2.467027,0.240221
6,2.462600,2.441635,0.248584
7,2.435200,2.422707,0.256869
8,2.421700,2.384675,0.268045
9,2.393900,2.355874,0.277235
10,2.376900,2.337508,0.284656


KeyboardInterrupt: 

In [ ]:
from transformers import Trainer, TrainingArguments
    
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=1e-4,
    num_train_epochs=200,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch",
    bf16=True,
    push_to_hub=True,
    hub_model_id="mlewand/PROTBERT_LM-L8_hid512",
    eval_accumulation_steps = 8 # Transfer GPU tensors during validation to RAM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    preprocess_logits_for_metrics= preprocess_logits_for_metrics
)

trainer.train()

In [ ]:
from  transformers import pipeline

model=model.to("cpu")
p = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
seq = train_sentences.iloc[0]
seq = seq[:-1]
seq += '<mask>'

p(seq)

In [ ]:
!pip install torchinfo
from torchinfo import summary 

In [ ]:
summary(model)

In [ ]:
# your logic for gettin thedata

In [ ]:
def getModel1():
    return keras.Sequential(
    [
        // model .....
    ]
)

model = getModel1()
print(model.summary())
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.1)